In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np

# proFSA
import fsa.load as load
import fsa.spatial as spatial
import fsa.sequence as seq
import fsa.elastic as elastic

In [2]:
# Choice tree to costumize calculations
WEIGHT_METHOD = ["intersect", "linear", "minimal"]
WEIGHT_METHOD_PARAMS = {"intersect": [10.],
                        "linear": [10., 10.1],
                        "minimal": [3]}
SPATIAL_ALIGNMENT = [True, False] 

# default choices
weight_method = WEIGHT_METHOD[0]
weight_method_params = WEIGHT_METHOD_PARAMS[weight_method]
spatial_alignment = SPATIAL_ALIGNMENT[0]

In [3]:
# Input
rel_pdb, def_pdb = '6FKI', '6FKF'

# Load
rel_pps = load.single_structure(rel_pdb)
def_pps = load.single_structure(def_pdb)

In [4]:
# Sequence alignment
com_res, rel_dict, def_dict = seq.pairwise_alignment(rel_pps, def_pps)

In [5]:
# Spatial alignment
if spatial_alignment:
    spatial.align_structures(rel_pps,
                             def_pps,
                             common_res = com_res,
                             rel_dict = rel_dict,
                             def_dict = def_dict)

In [6]:
# Coordinates
rel_xyz, rel_labels = load.coordinates(rel_pps,
                                       com_res,
                                       rel_dict)

def_xyz, def_labels = load.coordinates(def_pps,
                                       com_res,
                                       def_dict)

In [7]:
# Strain prepare
weights = elastic.compute_weights_fast([rel_xyz, def_xyz],
                                       method = weight_method,
                                       parameters = weight_method_params)

F = elastic.deformation_gradient_fast(weights, rel_xyz, def_xyz)


In [8]:
# Strain results
rot_angle, rot_axis = elastic.rotations(F)
rot_axis = spatial.vector_to_polar(rel_xyz, rot_axis)
_, gam_n = elastic.lagrange_strain(F)
stretches, stretch_axis = elastic.principal_stretches_from_g(gam_n)

In [9]:
# Save to PDB
strain_cluster_dict = {}
for label, st in zip(rel_labels, np.abs(stretches[:, 0])):
    strain_cluster_dict[label[0]] = st

load.save_bfactor(name=rel_pdb,
                  bfact=strain_cluster_dict,
                  ext='_st_quant_L0')

In [10]:
# Create dictionaries and files with principal stretches and rotation angles
var_values = [np.abs(stretches[:, 0]), 
              np.abs(stretches[:, 1]), 
              np.abs(stretches[:, 2]), 
              rot_angle]

var_names = ["lambda_1", 
             "lambda_2", 
             "lambda_3", 
             "rotation_angle"]

for name, value in zip(var_names, var_values):
    filename = rel_pdb + "-" + def_pdb + "-" + name
    var_cluster_dict = {}
    
    with open(filename + ".txt", "w") as f:
        for label, var in zip(rel_labels, value):
            var_cluster_dict[label[0]] = var
            f.write(str(label[0]) + ": " + str(var) + "\n")